In [ ]:
# 调试tensorboard

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

mnist = input_data.read_data_sets("MINIST_data", one_hot=True)

max_step = 1001


# 每个批次的大小 每次放入100张图片
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size  # 整除


# 参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean) # 记录值并且给一个名字
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)


with tf.name_scope('inputinput'):
    x = tf.placeholder(tf.float32, [None, 784], name = 'x-input')
    y = tf.placeholder(tf.float32, [None, 10], name = 'y-input')
    keep_prob = tf.placeholder(tf.float32)
    lr = tf.Variable(0.001, dtype = tf.float32)

with tf.name_scope('layer1'):
    with tf.name_scope('weight1'):
        Weight_L1 = tf.Variable(tf.truncated_normal([784,10], stddev=0.1), name='W')
        variable_summaries(Weight_L1)
    with tf.name_scope('biases1'):
        biases_L1 = tf.Variable(tf.zeros([1,10]) + 0.1, name='b')
        variable_summaries(biases_L1)
    with tf.name_scope('wx_plus_b1'):
        Wx_plus_b_L1 = tf.matmul(x, Weight_L1) + biases_L1
    with tf.name_scope('tanh1'):
        L1 = tf.nn.tanh(Wx_plus_b_L1)
    with tf.name_scope('dropout1'):
        L1_drop = tf.nn.dropout(L1, keep_prob)

with tf.name_scope('layer2'):
    with tf.name_scope('weight2'):
        Weight_L2 = tf.Variable(tf.truncated_normal([10,10], stddev=0.1))
    with tf.name_scope('biases2'):
        biases_L2 = tf.Variable(tf.zeros([1,10]) + 0.1)
    with tf.name_scope('wx_plus_b2'):
        Wx_plus_b_L2 = tf.matmul(L1_drop, Weight_L2) + biases_L2
    with tf.name_scope('tanh2'):
        L2 = tf.nn.tanh(Wx_plus_b_L2)
    with tf.name_scope('dropout2'):
        L2_drop = tf.nn.dropout(L2, keep_prob)
        
with tf.name_scope('layer3'):
    with tf.name_scope('weight3'):
        Weight_L3 = tf.Variable(tf.truncated_normal([10,10], stddev=0.1))
    with tf.name_scope('biases3'):
        biases_L3 = tf.Variable(tf.zeros([1,10]) + 0.1)
    with tf.name_scope('wx_plus_b3'):
        Wx_plus_b_L3 = tf.matmul(L2_drop, Weight_L3) + biases_L3
    with tf.name_scope('tanh3'):
        L3 = tf.nn.tanh(Wx_plus_b_L3)
    with tf.name_scope('dropout3'):
        L3_drop = tf.nn.dropout(L3, keep_prob)

with tf.name_scope('layer4'):
    Weight_L4 = tf.Variable(tf.truncated_normal([10,10], stddev=0.1))
    biases_L4 = tf.Variable(tf.zeros([1,10]) + 0.1)
    Wx_plus_b_L4 = tf.matmul(L3_drop, Weight_L4) + biases_L4


prediction = tf.nn.softmax(Wx_plus_b_L4)


# 二次代价函数
#loss = tf.reduce_mean(tf.square(y-prediction))
# 交叉熵代价函数
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    tf.summary.scalar('loss', loss) # 只有一个值
with tf.name_scope('TRAIN'):
    train_step = tf.train.AdamOptimizer(lr).minimize(loss)

init = tf.global_variables_initializer()


with tf.name_scope('accuracy'):
    with tf.name_scope('hahaha1'):
        correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction,1))
    with tf.name_scope('acccccc'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('accuracy', accuracy)
        
# 合并所有检测指标summary
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs/', sess.graph)
    for epoch in range(51):
        sess.run(tf.assign(lr, 0.001*(0.95**epoch)))
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            summary,_ = sess.run([merged, train_step], feed_dict={x:batch_xs, y:batch_ys, keep_prob:1.0}) # 同时训练和反馈
           
        writer.add_summary(summary, epoch)
        test_acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})
        train_acc = sess.run(accuracy, feed_dict={x:mnist.train.images, y:mnist.train.labels, keep_prob:1.0})
        print("Iter " +str(epoch) + " , testing accuracy " + str(test_acc) + "training accuracy" + str(train_acc))

In [2]:
# 第五课的程序 直接copy
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector
#载入数据集
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
#运行次数
max_steps = 1001
#图片数量
image_num = 1600
#文件路径
DIR = "E:/Calculations/"

#定义会话
sess = tf.Session()

#载入图片
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]),trainable=False, name='embedding')

#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean) #平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev', stddev) #标准差
        tf.summary.scalar('max', tf.reduce_max(var)) #最大值
        tf.summary.scalar('min', tf.reduce_min(var)) #最小值
        tf.summary.histogram('histogram', var) #直方图
        
#命名空间
with tf.name_scope('input'):
    #这里的none表示第一个维度可以是任意的长度
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    #正确的标签
    y = tf.placeholder(tf.float32, [None, 10], name='y-input')
    
#显示图片
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)
    
with tf.name_scope('layer'):
    #创建一个简单神经网络
    with tf.name_scope('weights'):
        W = tf.Variable(tf.zeros([784, 10]), name='W')
        variable_summaries(W)
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10]), name='b')
        variable_summaries(b)
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x,W) + b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(wx_plus_b)
    
with tf.name_scope('loss'):
    #交叉熵代价函数
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    tf.summary.scalar('loss', loss)
with tf.name_scope('train'):
    #使用梯度下降法
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [4]:
#初始化变量
sess.run(tf.global_variables_initializer())

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        #结果存放在一个布尔型列表中
        correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction, 1)) #argmax返回一维张量中最大值所在位置
    with tf.name_scope('accuracy'):
        #求准确率
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #把correct_prediction变为float32类型
        tf.summary.scalar('accuracy', accuracy)
    
#产生matdata文件
if tf.gfile.Exists(DIR + 'projector/projector/metadata.tsv'):
    tf.gfile.DeleteRecursively(DIR + 'projector/projector/metadata.tsv')
with open(DIR + 'projector/projector/metadata.tsv', 'w') as f:
    labels = sess.run(tf.argmax(mnist.test.labels[:],1))
    for i in range(image_num):
        f.write(str(labels[i]) + '\n') #存入对应图片的label

#合并所有的summary
merged = tf.summary.merge_all()

projector_writer = tf.summary.FileWriter(DIR + 'projector/projector', sess.graph)
saver = tf.train.Saver()
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = DIR + 'projector/projector/metadata.tsv'
embed.sprite.image_path = DIR + 'projector/data/mnist_1400x1064.png'
embed.sprite.single_image_dim.extend([28, 28])
projector.visualize_embeddings(projector_writer, config)

for i in range(max_steps):
    #每个批次100个样本
    batch_xs,batch_ys = mnist.train.next_batch(100) # 每次训练100张图片
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    summary,_ = sess.run([merged, train_step],feed_dict={x:batch_xs,y:batch_ys},options=run_options,run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata, 'step%03d' % i)
    projector_writer.add_summary(summary, i)
    
    if i%100 == 0:
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter" + str(i) + ", Testing Accuracy=" + str(acc))
        
saver.save(sess, DIR + 'projector/projector/a_model.ckpt', global_step=max_steps) # 保存训练好的模型
projector_writer.close()
sess.close()


Iter0, Testing Accuracy=0.2699
Iter100, Testing Accuracy=0.8057
Iter200, Testing Accuracy=0.8207
Iter300, Testing Accuracy=0.8268
Iter400, Testing Accuracy=0.8309
Iter500, Testing Accuracy=0.8677
Iter600, Testing Accuracy=0.8879
Iter700, Testing Accuracy=0.8981
Iter800, Testing Accuracy=0.9018
Iter900, Testing Accuracy=0.9041
Iter1000, Testing Accuracy=0.9049
